In [1]:
from pathlib import Path
import pdfplumber
import pandas as pd


In [9]:

eye_filepath = Path('..') / 'data' / 'raw' / 'Eye_test_report (1).pdf'


In [10]:
try:
    with pdfplumber.open(eye_filepath) as pdf:
        print("PDF opened successfully!")
        print(f"Number of pages: {len(pdf.pages)}")
        print(f"PDF metadata: {pdf.metadata}")
except Exception as e:
    print(f"Error opening PDF: {e}")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


PDF opened successfully!
Number of pages: 2
PDF metadata: {'Title': 'DRP report.pdf', 'Producer': 'macOS Version 12.7.6 (Build 21H1320) Quartz PDFContext', 'CreationDate': "D:20250603080158Z00'00'", 'ModDate': "D:20250603080158Z00'00'"}


In [11]:
# Check what's on each page
with pdfplumber.open(eye_filepath) as pdf:
    for i, page in enumerate(pdf.pages):
        print(f"\n=== PAGE {i+1} ===")
        
        # Check for text
        text = page.extract_text()
        if text:
            print(f"Text found: {len(text)} characters")
            print("First 200 characters:")
            print(text[:200])
        else:
            print("No text found")
        
        # Check for tables
        tables = page.extract_tables()
        print(f"Tables found: {len(tables)}")
        
        if tables:
            for j, table in enumerate(tables):
                print(f"  Table {j+1}: {len(table)} rows, {len(table[0]) if table else 0} columns")
                # Show first few rows
                print("  Preview:")
                for row in table[:3]:  # First 3 rows
                    print(f"    {row}")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox



=== PAGE 1 ===
No text found
Tables found: 0

=== PAGE 2 ===
No text found
Tables found: 0


In [12]:
# Let's see what pdfplumber can detect
with pdfplumber.open(eye_filepath) as pdf:
    for i, page in enumerate(pdf.pages):
        print(f"\nPage {i+1}:")
        print(f"  Images: {len(page.images)}")
        print(f"  Characters: {len(page.chars)}")
        print(f"  Lines: {len(page.lines)}")
        print(f"  Rectangles: {len(page.rects)}")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox



Page 1:
  Images: 2
  Characters: 0
  Lines: 6
  Rectangles: 3

Page 2:
  Images: 2
  Characters: 0
  Lines: 0
  Rectangles: 0


## Extracting using Nanonets OCR

In [1]:
from pathlib import Path
from pdf2image import convert_from_path
from transformers import AutoModelForImageTextToText, AutoTokenizer, AutoProcessor


/Users/mac/miniconda3/envs/med/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_path = "nanonets/Nanonets-OCR-s"

model = AutoModelForImageTextToText.from_pretrained(
    model_path, 
    torch_dtype="auto", 
    device_map="auto", 
    # attn_implementation="flash_attention_2"
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_path)
processor = AutoProcessor.from_pretrained(model_path)


def ocr_page_with_nanonets_s(image, model, processor, max_new_tokens=4096):
    prompt = """Extract the text from the above document as if you were reading it naturally. Return the tables in html format.
    Return the equations in LaTeX representation. If there is an image in the document and image caption is not present,
    add a small description of the image inside the <img></img> tag; otherwise, add the image caption inside <img></img>.
    Watermarks should be wrapped in brackets. Ex: <watermark>OFFICIAL COPY</watermark>. Page numbers should be wrapped in brackets.
    Ex: <page_number>14</page_number> or <page_number>9/22</page_number>. Prefer using ☐ and ☑ for check boxes."""

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": prompt},
        ]},
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[image], padding=True, return_tensors="pt")
    inputs = inputs.to(model.device)
    
    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output_ids)]
    
    output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return output_text[0]

# Convert PDF to images
eye_filepath = Path('..') / 'data' / 'raw' / 'Eye_test_report (1).pdf'
images = convert_from_path(eye_filepath, dpi=300)

# OCR each page
for i, image in enumerate(images):
    print(f"\n--- Page {i+1} ---")
    text = ocr_page_with_nanonets_s(image, model, processor, max_new_tokens=3000)
    print(text)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.71it/s]
Some parameters are on the meta device because they were offloaded to the disk and cpu.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.



--- Page 1 ---


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


## Extract with OpenAI

In [ ]:
extraction_agent = Agent()